### CÓDIGO COMÚN

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, MinMaxScaler
import statistics as stat

print('Librerías importadas correctamente')

Librerías importadas correctamente


In [2]:
bmw = pd.read_csv('/Users/davidlarre/Desktop/PROYECTOS/ENTREGABLE 1/data/bmw_pricing.csv')
df_bmw = bmw.copy()

In [3]:
# Se modifica el dtype de las columnas 'fecha_registro' y 'fecha_venta' a datetime
for i in ['fecha_registro','fecha_venta']:
    df_bmw[i]=pd.to_datetime(df_bmw[i])
# Comprobamos el cambio de dtype
print(df_bmw[['fecha_registro','fecha_venta']].dtypes)

# Se modifica el dtype de las columnas 'fecha_registro' y 'fecha_venta' a datetime
for i in ['fecha_registro','fecha_venta']:
    bmw[i]=pd.to_datetime(df_bmw[i])
# Comprobamos el cambio de dtype
print(bmw[['fecha_registro','fecha_venta']].dtypes)

fecha_registro    datetime64[ns]
fecha_venta       datetime64[ns]
dtype: object
fecha_registro    datetime64[ns]
fecha_venta       datetime64[ns]
dtype: object


In [4]:
# Consideramos que las fechas completas no aportan información relevante, por lo que vamos a crear nuevas columnas solo con el año
df_bmw['año_registro'] = df_bmw['fecha_registro'].dt.year
df_bmw['año_venta'] = df_bmw['fecha_venta'].dt.year
# Comprobamos que se han creado correctamente
df_bmw[['año_registro','año_venta']].head(10)

# Consideramos que las fechas completas no aportan información relevante, por lo que vamos a crear nuevas columnas solo con el año
bmw['año_registro'] = bmw['fecha_registro'].dt.year
bmw['año_venta'] = bmw['fecha_venta'].dt.year
# Comprobamos que se han creado correctamente
bmw[['año_registro','año_venta']].head(10)

,año_registro,año_venta
0,2012.0,2018.0
1,NaN,2018.0
2,2012.0,2018.0
3,NaN,2018.0
4,NaN,2018.0
5,2011.0,2018.0
6,NaN,2018.0
7,2009.0,2018.0
8,NaN,2018.0
9,NaN,2018.0


In [5]:
# Creamos una función para crear un DataFrame con la cantidad de nulos, el porcentaje de nulos y los valores únicos de cada columna. Ordenaremos el df por el porcentaje de nulos
def tabla_nulos(dataset):
    nulos = dataset.isnull().sum()
    porcentaje = dataset.isnull().mean() * 100
    unicos = dataset.nunique()
    
    df_nulos = pd.DataFrame({
        'NULOS': nulos,
        '% NULOS': porcentaje,
        'VALORES UNICOS': unicos
        }).sort_values(by='% NULOS', ascending=False)
    return df_nulos
    
tabla_nulos(df_bmw)
tabla_nulos(bmw)

,NULOS,% NULOS,VALORES UNICOS
asientos_traseros_plegables,3391,70.018584,2
año_registro,2423,50.030973,23
fecha_registro,2423,50.030973,180
tipo_coche,1460,30.146603,8
marca,970,20.028908,1
bluetooth,728,15.032005,2
alerta_lim_velocidad,728,15.032005,2
aire_acondicionado,486,10.035102,2
color,445,9.188520,10
precio,6,0.123890,437


In [6]:
# Creamos un diccionario para almacenar el porcentaje que tiene cada valor respecto al total de cada columna
distribuciones = {}
for i in df_bmw:
    distribuciones[i] = df_bmw[i].value_counts(normalize=True)

In [7]:
def obtener_moda(x):
    moda = x.mode()
    if not moda.empty:
        return moda.iloc[0]
    else:
        return np.nan

In [8]:
# 1. Cargar el dataset original
df_bmw_original = pd.read_csv('/Users/davidlarre/Desktop/PROYECTOS/ENTREGABLE 1/bmw_original.csv')

## COMPROBACION DE LA IMPUTACIÓN DE TIPO COCHE

### VERSIÓN 1 - 72,05% 

In [9]:
df_bmw['edad_coche'] = df_bmw['año_venta'] - df_bmw['año_registro']

In [10]:
df_bmw

,marca,modelo,km,potencia,fecha_registro,tipo_gasolina,color,tipo_coche,volante_regulable,aire_acondicionado,...,asientos_traseros_plegables,elevalunas_electrico,bluetooth,gps,alerta_lim_velocidad,precio,fecha_venta,año_registro,año_venta,edad_coche
0,NaN,118,140411.0,100.0,2012-02-01,diesel,black,NaN,True,True,...,NaN,True,NaN,True,NaN,11300.0,2018-01-01,2012.0,2018.0,6.0
1,BMW,M4,13929.0,317.0,NaT,petrol,grey,convertible,True,True,...,NaN,False,True,True,True,69700.0,2018-02-01,NaN,2018.0,NaN
2,BMW,320,183297.0,120.0,2012-04-01,diesel,white,NaN,False,False,...,NaN,True,False,True,False,10200.0,2018-02-01,2012.0,2018.0,6.0
3,BMW,420,128035.0,135.0,NaT,diesel,red,convertible,True,True,...,NaN,True,True,True,NaN,25100.0,2018-02-01,NaN,2018.0,NaN
4,BMW,425,97097.0,160.0,NaT,diesel,silver,NaN,True,True,...,False,False,True,True,True,33400.0,2018-04-01,NaN,2018.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4838,BMW,218 Gran Tourer,39743.0,110.0,NaT,diesel,black,NaN,False,True,...,NaN,False,False,True,False,14600.0,2018-08-01,NaN,2018.0,NaN
4839,BMW,218 Active Tourer,49832.0,100.0,2015-06-01,diesel,grey,NaN,False,True,...,NaN,False,False,True,True,17500.0,2018-08-01,2015.0,2018.0,3.0
4840,BMW,218 Gran Tourer,19633.0,110.0,2015-10-01,diesel,grey,van,False,True,...,NaN,False,False,True,True,17000.0,2018-09-01,2015.0,2018.0,3.0
4841,BMW,218 Active Tourer,27920.0,110.0,2016-04-01,diesel,brown,van,True,True,...,False,False,False,True,True,22700.0,2018-09-01,2016.0,2018.0,2.0


In [11]:
tabla_nulos(bmw).loc[['tipo_coche']]

,NULOS,% NULOS,VALORES UNICOS
tipo_coche,1460,30.146603,8


In [12]:
df_bmw['tipo_coche'].unique()

array([nan, 'convertible', 'coupe', 'estate', 'hatchback', 'sedan',
       'subcompact', 'suv', 'van'], dtype=object)

In [13]:
moda_paso_1 = df_bmw.groupby(['modelo', 'potencia', 'edad_coche'])['tipo_coche'].agg(obtener_moda).reset_index()
moda_paso_1 = moda_paso_1.rename(columns={'tipo_coche': 'MODA_PASO_1'})
df_bmw = df_bmw.merge(moda_paso_1, on=['modelo', 'potencia', 'edad_coche'], how='left')
df_bmw['tipo_coche'] = df_bmw['tipo_coche'].fillna(df_bmw['MODA_PASO_1'])
df_bmw.drop(columns=['MODA_PASO_1'], inplace=True)
tabla_nulos(df_bmw).loc[['tipo_coche']]
moda_paso_2 = df_bmw.groupby(['modelo', 'potencia'])['tipo_coche'].agg(obtener_moda).reset_index()
moda_paso_2 = moda_paso_2.rename(columns={'tipo_coche': 'MODA_PASO_2'})
df_bmw = df_bmw.merge(moda_paso_2, on=['modelo', 'potencia'], how='left')
df_bmw['tipo_coche'] = df_bmw['tipo_coche'].fillna(df_bmw['MODA_PASO_2'])
df_bmw.drop(columns=['MODA_PASO_2'], inplace=True)
tabla_nulos(df_bmw).loc[['tipo_coche']]
moda_paso_2 = df_bmw.groupby(['modelo'])['tipo_coche'].agg(obtener_moda).reset_index()
moda_paso_2 = moda_paso_2.rename(columns={'tipo_coche': 'MODA_PASO_2'})
df_bmw = df_bmw.merge(moda_paso_2, on=['modelo'], how='left')
df_bmw['tipo_coche'] = df_bmw['tipo_coche'].fillna(df_bmw['MODA_PASO_2'])
df_bmw.drop(columns=['MODA_PASO_2'], inplace=True)
moda_general = df_bmw['tipo_coche'].mode().iloc[0]
df_bmw['tipo_coche'] = df_bmw['tipo_coche'].fillna(moda_general)
tabla_nulos(df_bmw).loc[['tipo_coche']]


,NULOS,% NULOS,VALORES UNICOS
tipo_coche,0,0.0,8


In [14]:
# 2. Asegurarnos de que ambos dataframes tienen el mismo índice (o una columna común para alinear)
# Puedes hacer merge si hay un ID único, pero si no, asumimos que el orden es el mismo

# 3. Identificar las filas que originalmente tenían nulos en 'tipo_coche'
mask_nulos_originales = bmw['tipo_coche'].isnull()

# 4. Filtrar solo esas filas para comparar
tipo_coche_imputado = df_bmw.loc[mask_nulos_originales, 'tipo_coche']
tipo_coche_real = df_bmw_original.loc[mask_nulos_originales, 'tipo_coche']

# 5. Calcular aciertos
aciertos = (tipo_coche_imputado == tipo_coche_real).sum()
total = mask_nulos_originales.sum()
porcentaje_acierto = (aciertos / total) * 100 if total > 0 else None

# 6. Mostrar resultados
print(f"Aciertos: {aciertos} de {total} imputaciones")
print(f"Porcentaje de acierto: {porcentaje_acierto:.2f}%")

Aciertos: 1052 de 1460 imputaciones
Porcentaje de acierto: 72.05%


## COMPROBACION DE LA IMPUTACIÓN COLOR

### VERSIÓN 3 - 80% ACIERTO

In [15]:
def obtener_color_binario(x):
  if str(x) == 'nan':
    return np.nan
  elif x in ['black','grey','blue','white']:
    return 1
  else:
    return 0

In [16]:
df_bmw['color_estandar'] = df_bmw['color'].map(obtener_color_binario)

In [17]:
df_bmw_original['color_estandar'] = df_bmw_original['color'].map(obtener_color_binario)

In [18]:
df_bmw[['color','color_estandar']]

,color,color_estandar
0,black,1.0
1,grey,1.0
2,white,1.0
3,red,0.0
4,silver,0.0
...,...,...
4838,black,1.0
4839,grey,1.0
4840,grey,1.0
4841,brown,0.0


In [19]:
# guardemos los valores de la moda para color_estandar en tablas diferentes por niveles de agrupación
tabla_moda_color_estandar = df_bmw[~(df_bmw['color_estandar'].isnull())].groupby(['modelo','tipo_coche','potencia'])['color_estandar'].agg([stat.mode])

In [20]:
tabla_moda_color_estandar_por_modelo_y_tipo_coche = df_bmw[~(df_bmw['color_estandar'].isnull())].groupby(['modelo','tipo_coche'])['color_estandar'].agg([stat.mode])

In [21]:
tabla_moda_color_estandar_por_modelo = df_bmw[~(df_bmw['color_estandar'].isnull())].groupby(['modelo'])['color_estandar'].agg([stat.mode])

In [22]:
tabla_moda_color_estandar_por_tipo_coche = df_bmw[~(df_bmw['color_estandar'].isnull())].groupby(['tipo_coche'])['color_estandar'].agg([stat.mode])

In [23]:
len(tabla_moda_color_estandar_por_modelo)==len(df_bmw['modelo'].unique())

False

In [24]:
#falta un modelo que es unico en todo el dataset y tiene color_estandar nulo
len(df_bmw['modelo'].unique())

77

In [25]:
# reemplacemoslos nulos en color_estandar con la moda por modelo, tipo_coche y potencia. Cuando no es posible usemos la moda solo por modelo y tipo_coche. Si tampoco esta existe
# usemos la moda por modelo, e si tampoco esta existe le ponemos la moda de color_estandar para tipo_coche, osea 1

for i in df_bmw[df_bmw['color_estandar'].isnull()].index:
  modelo = df_bmw.loc[i,'modelo']
  tipo_coche = df_bmw.loc[i,'tipo_coche']
  potencia = df_bmw.loc[i,'potencia']
 # existe la moda por modelo, tipo_coche y potencia?
  if (modelo,tipo_coche,potencia) in tabla_moda_color_estandar.index:
    color_estandar = tabla_moda_color_estandar.loc[(modelo,tipo_coche,potencia)].values[0]
 # si no, existe la moda solo por modelo y tipo_coche?
  elif (modelo,tipo_coche) in tabla_moda_color_estandar_por_modelo_y_tipo_coche.index:
    color_estandar = tabla_moda_color_estandar_por_modelo_y_tipo_coche.loc[(modelo,tipo_coche)].values[0]
 # si no, existe la moda solo por modelo?
  elif (modelo) in tabla_moda_color_estandar_por_modelo.index:
    color_estandar = tabla_moda_color_estandar_por_modelo.loc[(modelo)].values[0]
  else:
    color_estandar = tabla_moda_color_estandar_por_tipo_coche.loc[(tipo_coche)].values[0]
  df_bmw.loc[i,'color_estandar'] = color_estandar

In [26]:
tabla_nulos(df_bmw)

,NULOS,% NULOS,VALORES UNICOS
asientos_traseros_plegables,3391,70.018584,2
fecha_registro,2423,50.030973,180
edad_coche,2423,50.030973,25
año_registro,2423,50.030973,23
marca,970,20.028908,1
bluetooth,728,15.032005,2
alerta_lim_velocidad,728,15.032005,2
aire_acondicionado,486,10.035102,2
color,445,9.188520,10
precio,6,0.123890,437


In [27]:
df_bmw_original['color_estandar'].value_counts(normalize=True)

color_estandar
1    0.837497
0    0.162503
Name: proportion, dtype: float64

In [28]:
df_bmw.head()

,marca,modelo,km,potencia,fecha_registro,tipo_gasolina,color,tipo_coche,volante_regulable,aire_acondicionado,...,elevalunas_electrico,bluetooth,gps,alerta_lim_velocidad,precio,fecha_venta,año_registro,año_venta,edad_coche,color_estandar
0,NaN,118,140411.0,100.0,2012-02-01,diesel,black,coupe,True,True,...,True,NaN,True,NaN,11300.0,2018-01-01,2012.0,2018.0,6.0,1.0
1,BMW,M4,13929.0,317.0,NaT,petrol,grey,convertible,True,True,...,False,True,True,True,69700.0,2018-02-01,NaN,2018.0,NaN,1.0
2,BMW,320,183297.0,120.0,2012-04-01,diesel,white,sedan,False,False,...,True,False,True,False,10200.0,2018-02-01,2012.0,2018.0,6.0,1.0
3,BMW,420,128035.0,135.0,NaT,diesel,red,convertible,True,True,...,True,True,True,NaN,25100.0,2018-02-01,NaN,2018.0,NaN,0.0
4,BMW,425,97097.0,160.0,NaT,diesel,silver,coupe,True,True,...,False,True,True,True,33400.0,2018-04-01,NaN,2018.0,NaN,0.0


In [29]:
# despues de la sostitucion de los nulos se ha mantenido la distribucion inicial de los valores ----> OK

In [30]:
# 1. Identificar las filas que originalmente tenían nulos en 'color'
nulos_originales = df_bmw['color'].isnull()

# 2. Filtrar solo esas filas para comparar
color_imputado = df_bmw.loc[nulos_originales, 'color_estandar'] # df donde se han imputado nulos
color_real = df_bmw_original.loc[nulos_originales, 'color_estandar'] # copia del df original sin modificaciones

# 3. Calcular aciertos
aciertos = (color_imputado == color_real).sum()
total = nulos_originales.sum()
porcentaje_acierto = (aciertos / total) * 100 if total > 0 else None

# 4. Mostrar resultados
print(f"Aciertos: {aciertos} de {total} imputaciones")
print(f"Porcentaje de acierto: {porcentaje_acierto:.2f}%")

Aciertos: 356 de 445 imputaciones
Porcentaje de acierto: 80.00%


## COMPROBACIÓN DE LA IMPUTACIÓN EDAD COCHE

### VERSIÓN 2 - 83,37% AHORA 83,24%

In [31]:
for i in ['fecha_registro','fecha_venta']:
    df_bmw[i]=pd.to_datetime(df_bmw[i])

for i in ['fecha_registro','fecha_venta']:
    bmw[i]=pd.to_datetime(bmw[i])
    
for i in ['fecha_registro','fecha_venta']:
    df_bmw_original[i]=pd.to_datetime(df_bmw_original[i])

# Consideramos que las fechas completas no aportan información relevante, por lo que vamos a crear nuevas columnas solo con el año
df_bmw['año_registro'] = df_bmw['fecha_registro'].dt.year
df_bmw['año_venta'] = df_bmw['fecha_venta'].dt.year
df_bmw['edad'] = df_bmw['año_venta'] - df_bmw['año_registro']

bmw['año_registro'] = bmw['fecha_registro'].dt.year
bmw['año_venta'] = bmw['fecha_venta'].dt.year
bmw['edad'] = bmw['año_venta'] - bmw['año_registro']

df_bmw_original['año_registro'] = df_bmw_original['fecha_registro'].dt.year
df_bmw_original['año_venta'] = df_bmw_original['fecha_venta'].dt.year
df_bmw_original['edad'] = df_bmw_original['año_venta'] - df_bmw_original['año_registro']

In [32]:
tabla_nulos(df_bmw).loc[['edad']]

,NULOS,% NULOS,VALORES UNICOS
edad,2423,50.030973,25


In [33]:
df_bmw['edad'].value_counts()

edad
 5.0     757
 4.0     635
 6.0     435
 3.0     162
 7.0     116
 8.0      51
 2.0      46
 10.0     43
 9.0      39
 12.0     29
 11.0     29
 13.0     28
 14.0     10
 17.0      8
 15.0      8
 1.0       5
 16.0      4
 21.0      4
 18.0      3
 19.0      3
-1.0       1
-5.0       1
 23.0      1
 28.0      1
 24.0      1
Name: count, dtype: int64

In [34]:
# Definir los rangos de edad y los nuevos valores
rangos = [0, 3, 6, 10, 15, 20, 25, 30]
nuevos_valores = ['1-3', '4-6', '7-10', '11-15', '16-20', '21-25', '>25']

# Aplicar la transformación con pd.cut
df_bmw['edad_transformada'] = pd.cut(df_bmw['edad'], bins=rangos, labels=nuevos_valores, right=True)
bmw['edad_transformada'] = pd.cut(bmw['edad'], bins=rangos, labels=nuevos_valores, right=True)
df_bmw_original['edad_transformada'] = pd.cut(df_bmw_original['edad'], bins=rangos, labels=nuevos_valores, right=True)

In [35]:
'''def get_clase_edad_coche(edad):
  if pd.isna(edad):
    return edad
  elif edad in range(1,4):
    return '1-3'
  elif edad in range(4,7):
    return '4-6'
  elif edad in range(7,11):
    return '7-10'
  elif edad in range(11,16):
    return '11-15'
  elif edad in range(16,21):
    return '16-20'
  elif edad in range(21,26):
    return '21-25'
  else:
    return '>25'''

"def get_clase_edad_coche(edad):\n  if pd.isna(edad):\n    return edad\n  elif edad in range(1,4):\n    return '1-3'\n  elif edad in range(4,7):\n    return '4-6'\n  elif edad in range(7,11):\n    return '7-10'\n  elif edad in range(11,16):\n    return '11-15'\n  elif edad in range(16,21):\n    return '16-20'\n  elif edad in range(21,26):\n    return '21-25'\n  else:\n    return '>25"

In [36]:
'''# redefiniendo edad_coche como la clase de edad del coche
df_bmw['edad'] = df_bmw['edad'].map(get_clase_edad_coche)
df_bmw_original['edad'] = df_bmw_original['edad'].map(get_clase_edad_coche)'''

"# redefiniendo edad_coche como la clase de edad del coche\ndf_bmw['edad'] = df_bmw['edad'].map(get_clase_edad_coche)\ndf_bmw_original['edad'] = df_bmw_original['edad'].map(get_clase_edad_coche)"

In [37]:
# guardemos los valores de la moda para edad_coche en tablas diferentes sugun el nivel de agrupación
tabla_moda_edad_coche = df_bmw[~(df_bmw['edad_transformada'].isnull())].groupby(['modelo','tipo_coche','potencia'])['edad_transformada'].agg([stat.mode])

In [38]:
tabla_moda_edad_coche_por_modelo_y_tipo_coche = df_bmw[~(df_bmw['edad_transformada'].isnull())].groupby(['modelo','tipo_coche'])['edad_transformada'].agg([stat.mode])

In [39]:
tabla_moda_edad_coche_por_modelo = df_bmw[~(df_bmw['edad_transformada'].isnull())].groupby(['km'])['edad_transformada'].agg([stat.mode])

In [40]:
# Reemplazemos los nulos en edad_coche con la moda por modelo, tipo_coche y potencia. Cuando no es posible usemos la moda solo por modelo y tipo_coche. Si tampoco esta existe
# usemos la moda por modelo, e si tampoco esta existe le ponemos la moda de edad_coche por el dataset osea '4-6'

for i in df_bmw[df_bmw['edad_transformada'].isnull()].index:
  modelo = df_bmw.loc[i,'modelo']
  tipo_coche = df_bmw.loc[i,'tipo_coche']
  potencia = df_bmw.loc[i,'potencia']
 # existe la moda por modelo, tipo_coche y potencia?
  if (modelo,tipo_coche,potencia) in tabla_moda_edad_coche.index:
    edad_transformada = tabla_moda_edad_coche.loc[(modelo,tipo_coche,potencia)].values[0]
 # si no, existe la moda solo por modelo y tipo_coche?
  elif (modelo,tipo_coche) in tabla_moda_edad_coche_por_modelo_y_tipo_coche.index:
    edad_transformada = tabla_moda_edad_coche_por_modelo_y_tipo_coche.loc[(modelo,tipo_coche)].values[0]
 # si no, existe la moda solo por modelo?
  elif (modelo) in tabla_moda_edad_coche_por_modelo.index:
    edad_transformada = tabla_moda_edad_coche_por_modelo.loc[(modelo)].values[0]
  else:
    edad_transformada = df_bmw['edad_transformada'].mode().values[0]
  df_bmw.loc[i,'edad_transformada'] = edad_transformada

In [41]:
# 1. Identificar las filas que originalmente tenían nulos en 'color'
nulos_originales = df_bmw['fecha_registro'].isnull()

# 2. Filtrar solo esas filas para comparar
edad_imputada = df_bmw.loc[nulos_originales, 'edad_transformada'] # df donde se han imputado nulos
edad_real = df_bmw_original.loc[nulos_originales, 'edad_transformada'] # copia del df original sin modificaciones

# 3. Calcular aciertos
aciertos = (edad_imputada == edad_real).sum()
total = nulos_originales.sum()
porcentaje_acierto = (aciertos / total) * 100 if total > 0 else None

# 4. Mostrar resultados
print(f"Aciertos: {aciertos} de {total} imputaciones")
print(f"Porcentaje de acierto: {porcentaje_acierto:.2f}%")

Aciertos: 2017 de 2423 imputaciones
Porcentaje de acierto: 83.24%


## COMPROBACIÓN DE LA IMPUTACIÓN AIRE ACONDICIONADO

### VERSIÓN 2 - 81, 07% modelo', 'edad / 'moddelo', potencia / modelo / moda general AHORA 81,07%

In [42]:
tabla_nulos(df_bmw).loc[['aire_acondicionado']]

,NULOS,% NULOS,VALORES UNICOS
aire_acondicionado,486,10.035102,2


In [43]:
moda_paso_1 = bmw.groupby(['modelo','edad'])['aire_acondicionado'].agg(obtener_moda).reset_index()
moda_paso_1 = moda_paso_1.rename(columns={'aire_acondicionado': 'MODA_PASO_1'})
df_bmw = df_bmw.merge(moda_paso_1, on=['modelo', 'edad'], how='left')
df_bmw['aire_acondicionado'] = df_bmw['aire_acondicionado'].fillna(df_bmw['MODA_PASO_1'])
df_bmw.drop(columns=['MODA_PASO_1'], inplace=True)

In [44]:
tabla_nulos(df_bmw).loc[['aire_acondicionado']]

,NULOS,% NULOS,VALORES UNICOS
aire_acondicionado,243,5.017551,2


In [45]:
moda_paso_1 = bmw.groupby(['modelo','potencia'])['aire_acondicionado'].agg(obtener_moda).reset_index()
moda_paso_1 = moda_paso_1.rename(columns={'aire_acondicionado': 'MODA_PASO_1'})
df_bmw = df_bmw.merge(moda_paso_1, on=['modelo','potencia'], how='left')
df_bmw['aire_acondicionado'] = df_bmw['aire_acondicionado'].fillna(df_bmw['MODA_PASO_1'])
df_bmw.drop(columns=['MODA_PASO_1'], inplace=True)

In [46]:
tabla_nulos(df_bmw).loc[['aire_acondicionado']]

,NULOS,% NULOS,VALORES UNICOS
aire_acondicionado,5,0.103242,2


In [47]:
moda_paso_1 = bmw.groupby(['modelo'])['aire_acondicionado'].agg(obtener_moda).reset_index()
moda_paso_1 = moda_paso_1.rename(columns={'aire_acondicionado': 'MODA_PASO_1'})
df_bmw = df_bmw.merge(moda_paso_1, on=['modelo'], how='left')
df_bmw['aire_acondicionado'] = df_bmw['aire_acondicionado'].fillna(df_bmw['MODA_PASO_1'])
df_bmw.drop(columns=['MODA_PASO_1'], inplace=True)

In [48]:
tabla_nulos(df_bmw).loc[['aire_acondicionado']]

,NULOS,% NULOS,VALORES UNICOS
aire_acondicionado,2,0.041297,2


In [49]:
# Calcular la moda de la columna 'bluetooth'
moda_bluetooth = bmw['aire_acondicionado'].mode().iloc[0]

# Rellenar los valores nulos con la moda
df_bmw['aire_acondicionado'] = df_bmw['aire_acondicionado'].fillna(moda_bluetooth)
tabla_nulos(df_bmw).loc[['aire_acondicionado']]

,NULOS,% NULOS,VALORES UNICOS
aire_acondicionado,0,0.0,2


In [50]:
# 1. Identificar las filas que originalmente tenían nulos en 'color'
nulos_originales = bmw['aire_acondicionado'].isnull()

# 2. Filtrar solo esas filas para comparar
edad_imputada = df_bmw.loc[nulos_originales, 'aire_acondicionado'] # df donde se han imputado nulos
edad_real = df_bmw_original.loc[nulos_originales, 'aire_acondicionado'] # copia del df original sin modificaciones

# 3. Calcular aciertos
aciertos = (edad_imputada == edad_real).sum()
total = nulos_originales.sum()
porcentaje_acierto = (aciertos / total) * 100 if total > 0 else None

# 4. Mostrar resultados
print(f"Aciertos: {aciertos} de {total} imputaciones")
print(f"Porcentaje de acierto: {porcentaje_acierto:.2f}%")

Aciertos: 394 de 486 imputaciones
Porcentaje de acierto: 81.07%


## COMPROBACIÓN DE LA IMPUTACIÓN BLUETOOTH

### VERSIÓN 2 - 79,26%  AHORA 78,16%

In [51]:
moda_paso_1 = bmw.groupby(['edad','tipo_coche','modelo','potencia', 'asientos_traseros_plegables', 'aire_acondicionado'])['bluetooth'].agg(obtener_moda).reset_index()
moda_paso_1 = moda_paso_1.rename(columns={'bluetooth': 'MODA_PASO_1'})

df_bmw = df_bmw.merge(moda_paso_1, on=['edad','tipo_coche','modelo','potencia', 'asientos_traseros_plegables', 'aire_acondicionado'], how='left')
df_bmw['bluetooth'] = df_bmw['bluetooth'].fillna(df_bmw['MODA_PASO_1'])
df_bmw.drop(columns=['MODA_PASO_1'], inplace=True)

In [52]:
tabla_nulos(df_bmw).loc[['bluetooth']]

,NULOS,% NULOS,VALORES UNICOS
bluetooth,661,13.648565,2


In [53]:
moda_paso_1 = bmw.groupby(['modelo', 'asientos_traseros_plegables', 'aire_acondicionado'])['bluetooth'].agg(obtener_moda).reset_index()
moda_paso_1 = moda_paso_1.rename(columns={'bluetooth': 'MODA_PASO_1'})

df_bmw = df_bmw.merge(moda_paso_1, on=['modelo', 'asientos_traseros_plegables', 'aire_acondicionado'], how='left')
df_bmw['bluetooth'] = df_bmw['bluetooth'].fillna(df_bmw['MODA_PASO_1'])
df_bmw.drop(columns=['MODA_PASO_1'], inplace=True)

In [54]:
tabla_nulos(df_bmw).loc[['bluetooth']]

,NULOS,% NULOS,VALORES UNICOS
bluetooth,502,10.365476,2


In [55]:
moda_paso_1 = bmw.groupby(['modelo','elevalunas_electrico'])['bluetooth'].agg(obtener_moda).reset_index()
moda_paso_1 = moda_paso_1.rename(columns={'bluetooth': 'MODA_PASO_1'})
df_bmw = df_bmw.merge(moda_paso_1, on=['modelo','elevalunas_electrico'], how='left')
df_bmw['bluetooth'] = df_bmw['bluetooth'].fillna(df_bmw['MODA_PASO_1'])
df_bmw.drop(columns=['MODA_PASO_1'], inplace=True)

In [56]:
tabla_nulos(df_bmw).loc[['bluetooth']]

,NULOS,% NULOS,VALORES UNICOS
bluetooth,5,0.103242,2


In [57]:
moda_paso_1 = bmw.groupby(['modelo'])['bluetooth'].agg(obtener_moda).reset_index()
moda_paso_1 = moda_paso_1.rename(columns={'bluetooth': 'MODA_PASO_1'})
df_bmw = df_bmw.merge(moda_paso_1, on=['modelo'], how='left')
df_bmw['bluetooth'] = df_bmw['bluetooth'].fillna(df_bmw['MODA_PASO_1'])
df_bmw.drop(columns=['MODA_PASO_1'], inplace=True)

In [58]:
tabla_nulos(df_bmw).loc[['bluetooth']]

,NULOS,% NULOS,VALORES UNICOS
bluetooth,2,0.041297,2


In [59]:
# Calcular la moda de la columna 'bluetooth'
moda_bluetooth = bmw['bluetooth'].mode().iloc[0]

# Rellenar los valores nulos con la moda
df_bmw['bluetooth'] = df_bmw['bluetooth'].fillna(moda_bluetooth)

In [60]:
tabla_nulos(df_bmw).loc[['bluetooth']]

,NULOS,% NULOS,VALORES UNICOS
bluetooth,0,0.0,2


In [61]:
# 1. Identificar las filas que originalmente tenían nulos en 'color'
nulos_originales = bmw['bluetooth'].isnull()

# 2. Filtrar solo esas filas para comparar
edad_imputada = df_bmw.loc[nulos_originales, 'bluetooth'] # df donde se han imputado nulos
edad_real = df_bmw_original.loc[nulos_originales, 'bluetooth'] # copia del df original sin modificaciones

# 3. Calcular aciertos
aciertos = (edad_imputada == edad_real).sum()
total = nulos_originales.sum()
porcentaje_acierto = (aciertos / total) * 100 if total > 0 else None

# 4. Mostrar resultados
print(f"Aciertos: {aciertos} de {total} imputaciones")
print(f"Porcentaje de acierto: {porcentaje_acierto:.2f}%")

Aciertos: 569 de 728 imputaciones
Porcentaje de acierto: 78.16%


## COMPROBACIÓN DE LA IMPUTACIÓN ALERTA LIMITE VELOCIDAD

### VERSIÓN 2 - 72,53% 'potencia', 'km / modelo', 'potencia / moda general

In [62]:
tabla_nulos(df_bmw).loc[['alerta_lim_velocidad']]

,NULOS,% NULOS,VALORES UNICOS
alerta_lim_velocidad,728,15.032005,2


In [63]:
distribuciones['alerta_lim_velocidad']

alerta_lim_velocidad
True     0.54435
False    0.45565
Name: proportion, dtype: float64

In [64]:
df_bmw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4843 entries, 0 to 4842
Data columns (total 24 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   marca                        3873 non-null   object        
 1   modelo                       4840 non-null   object        
 2   km                           4841 non-null   float64       
 3   potencia                     4842 non-null   float64       
 4   fecha_registro               2420 non-null   datetime64[ns]
 5   tipo_gasolina                4838 non-null   object        
 6   color                        4398 non-null   object        
 7   tipo_coche                   4843 non-null   object        
 8   volante_regulable            4839 non-null   object        
 9   aire_acondicionado           4843 non-null   bool          
 10  camara_trasera               4841 non-null   object        
 11  asientos_traseros_plegables  1452 non-null 

In [65]:
moda_paso_1 = df_bmw.groupby(['potencia', 'km'])['alerta_lim_velocidad'].agg(obtener_moda).reset_index()
moda_paso_1 = moda_paso_1.rename(columns={'alerta_lim_velocidad': 'MODA_PASO_1'})
df_bmw = df_bmw.merge(moda_paso_1, on=['potencia', 'km'], how='left')
df_bmw['alerta_lim_velocidad'] = df_bmw['alerta_lim_velocidad'].fillna(df_bmw['MODA_PASO_1'])
df_bmw.drop(columns=['MODA_PASO_1'], inplace=True)

In [66]:
tabla_nulos(df_bmw).loc[['alerta_lim_velocidad']]

,NULOS,% NULOS,VALORES UNICOS
alerta_lim_velocidad,723,14.928763,2


In [67]:
moda_paso_1 = df_bmw.groupby(['modelo', 'potencia'])['alerta_lim_velocidad'].agg(obtener_moda).reset_index()
moda_paso_1 = moda_paso_1.rename(columns={'alerta_lim_velocidad': 'MODA_PASO_1'})
df_bmw = df_bmw.merge(moda_paso_1, on=['modelo', 'potencia'], how='left')
df_bmw['alerta_lim_velocidad'] = df_bmw['alerta_lim_velocidad'].fillna(df_bmw['MODA_PASO_1'])
df_bmw.drop(columns=['MODA_PASO_1'], inplace=True)

In [68]:
tabla_nulos(df_bmw).loc[['alerta_lim_velocidad']]

,NULOS,% NULOS,VALORES UNICOS
alerta_lim_velocidad,13,0.268429,2


In [69]:
# Calcular la moda de la columna 'bluetooth'
moda_bluetooth = df_bmw['alerta_lim_velocidad'].mode().iloc[0]

# Rellenar los valores nulos con la moda
df_bmw['alerta_lim_velocidad'] = df_bmw['alerta_lim_velocidad'].fillna(moda_bluetooth)
tabla_nulos(df_bmw).loc[['alerta_lim_velocidad']]

,NULOS,% NULOS,VALORES UNICOS
alerta_lim_velocidad,0,0.0,2


In [70]:
# 1. Identificar las filas que originalmente tenían nulos en 'color'
nulos_originales = bmw['alerta_lim_velocidad'].isnull()

# 2. Filtrar solo esas filas para comparar
edad_imputada = df_bmw.loc[nulos_originales, 'alerta_lim_velocidad'] # df donde se han imputado nulos
edad_real = df_bmw_original.loc[nulos_originales, 'alerta_lim_velocidad'] # copia del df original sin modificaciones

# 3. Calcular aciertos
aciertos = (edad_imputada == edad_real).sum()
total = nulos_originales.sum()
porcentaje_acierto = (aciertos / total) * 100 if total > 0 else None

# 4. Mostrar resultados
print(f"Aciertos: {aciertos} de {total} imputaciones")
print(f"Porcentaje de acierto: {porcentaje_acierto:.2f}%")

Aciertos: 528 de 728 imputaciones
Porcentaje de acierto: 72.53%
